# Linear Regression

In [1]:
import numpy as np
import torch

## Training Data
The training data can be represented using 2 matrices: inputs and targets, each with one row per observation, and one column per variable.

In [36]:
# Input (temp, rainfall, humidity)
inputs = np.array([
    [73,67,43],
    [91,88,64],
    [87,134,58],
    [102,43,37],
    [69,96,70]],dtype = 'float32'
)

In [3]:
# Targets (apples,oranges)
targets = np.array([
    [56,70],
    [81,101],
    [119,133],
    [22,37],
    [103,119]
],dtype = 'float32'
)

We've seperated the input and target variables, because we'll operate on them seperately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors as follows:

IN Model:
    yield_apple = w11*temp + w12*rainfall + w13*humidity +b1
    yield_orange = w21*temp + w12*rainfall + w23*humidity + b2

As we need to figure out the values of the weights we start with random values then calculate the weights

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regerssion model from scratch
The weights and biases(w11,w12,...w23, b1 & b2) can also be represented as matrices, initialized as random values. The first row of w and first element of b are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [5]:
# Weights and biases
w = torch.randn(2,3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

# Note: The randn function uses gaussian distribution to get the values

tensor([[-0.6645,  0.1052, -0.3417],
        [ 0.6145, -0.4639, -0.1520]], requires_grad=True)
tensor([ 0.5957, -0.0583], requires_grad=True)


torch.randn creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviatrion 1.

Our model is simply a functiom that performs a matrix multiplication of the inputs and the weights [wtransposed] and adds the bias [b] (replicated for each observation).
X * W^T + b

We can define the model as follows:

In [6]:
def model(x):
    return x @ w.t() + b  # @ represents matrix multiplication in pytorch

.t method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [7]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-55.5584,   7.1792],
        [-72.4861,   5.3057],
        [-62.9381, -17.5811],
        [-75.3045,  37.0443],
        [-59.0750, -12.8353]], grad_fn=<AddBackward0>)


Now comparing the predictions of our model with teh actual targets.

In [8]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


We can see that there's a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to just work.

But as we have a model now we can start improving it to match the targets.

The weights a 2 by 3 matrix because we have 3 weights (w11,w12,w13).

## Loss function
Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:
    * Calculate the difference between the two matrices(preds and targets).
    * Square all elements of the difference matrix to remove negative values.
    * Calculate the average of the elements in the resulting matrix.

The result is a single number known as mean square error(MSE).


In [12]:
# MSE
diff = preds - targets
diff_sqr = diff *diff
torch.sum(diff_sqr)/ diff.numel()

tensor(15800.0264, grad_fn=<DivBackward0>)

In [15]:
# MSE loss
def mse(t1,t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

[torch.sum] returns the sum of all the elements in a tensor, and the .numel method returns the number of elements in  a tensor. Let's compute the mean squared error for the current predictions of our model.

In [16]:
# Compute loss (i.e compute M.S.E for the current set of predictions)
loss = mse(preds, targets)
print(loss)

tensor(15800.0264, grad_fn=<DivBackward0>)


Here's how we can interpret the result: On average, each element in the prediction differs from the actual target by about 138 (square root of the loss 19044). And that's pretty bad, considering the numbers we are trying to predict are themselves in the range 50-200. Also, the result is called the loss, because it indicates how bad the model is ate predicting the target variables. Lower the loss, better the model.

This tells us how much the model is lossing and lower the loss the better the model.

## Compute gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have requires_grad set to True. (see above)

In [17]:
# Compute gradients
loss.backward()

The gradients are stored in the .grad property of the respective tensors. Note that the derivative of loss w.r.t. the weights matrix is itself a matrix, with the same dimensions.

In [18]:
# Gradients for weights
print(w)
print(w.grad) # This is the derivative i.e the loss w.r.t a particular element.

tensor([[-0.6645,  0.1052, -0.3417],
        [ 0.6145, -0.4639, -0.1520]], requires_grad=True)
tensor([[-11809.5693, -13020.8369,  -8023.6094],
        [ -7097.3550,  -9092.4492,  -5357.2529]])


In [19]:
# Similarly for biases
print(b)
print(b.grad)

tensor([ 0.5957, -0.0583], requires_grad=True)
tensor([-141.2724,  -88.1774])


The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

If a gradient element is positive:
    * Increasing the element's value slightly will increase the loss.
    * Decreasing the element's value slightly will decrease the loss.
    
Eg: Plots w.r.t to one value eg(w11)


Note:
 The increase or decrease in the loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve the model.
 
Before we continue, we need to reset the gradients to zero by calling .zero() method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call .backward on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.
So, everytime we are done with any gradient related work we need to reset.

In [20]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


### Adjust weights and biases using gradient descent
We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:
    1. Generate predictions
    2. Calculate the loss
    3. Compute gradients w.r.t the weights and biases
    4. Adjust the weights by subtracting a small quantity
    5. Reset the gradients to zero
    
Let's implement the above step by step.

In [21]:
# Generate predictions (Step 1)
preds = model(inputs)
print(preds)

tensor([[-55.5584,   7.1792],
        [-72.4861,   5.3057],
        [-62.9381, -17.5811],
        [-75.3045,  37.0443],
        [-59.0750, -12.8353]], grad_fn=<AddBackward0>)


NOTE:
    The predictions are same as before, since we havenot made any changes to our model. The same holds true for the loss and gradients.

In [22]:
# Calculate the loss (Step 2)
loss = mse(preds, targets)
print(loss)

tensor(15800.0264, grad_fn=<DivBackward0>)


In [23]:
# Compute gradients (Step 3)
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-11809.5693, -13020.8369,  -8023.6094],
        [ -7097.3550,  -9092.4492,  -5357.2529]])
tensor([-141.2724,  -88.1774])


Finally, we update the weights and biases using the gradients computed above.

In [25]:
# Adjust weights and reset gradients
with torch.no_grad():
    w-= w.grad * 1e-5
    b-= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

Note:
    * We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate or modify gradients while upadting the weights and biases.
    * We multiply the gradients with really small number (10^5 in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the learning rate of the algorirthm. 
    * After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations. 

In [26]:
# The new weights and biases
print(w)
print(b)

tensor([[-0.5464,  0.2354, -0.2615],
        [ 0.6854, -0.3730, -0.0984]], requires_grad=True)
tensor([ 0.5985, -0.0566], requires_grad=True)


In [27]:
# Now again checking if we get a lower loss

preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(10926.4922, grad_fn=<DivBackward0>)


We can see that we have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent.

### Train for multiple epocs
To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [31]:
# Train for 500 epochs
for i in range(500):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [32]:
# Once again verifying that the loss is now lower:

# Calculate loss
preds = model(inputs)
loss = mse(preds,targets)

print(loss)  # i.e sqrt(14.3054). So we are off by just 14

tensor(14.3054, grad_fn=<DivBackward0>)


We can see that the loss is now much lower than what we started with. 
Now, Let's look at the models predictions and compare them with the targets.

In [33]:
# Predictions
preds

tensor([[ 57.6122,  70.9059],
        [ 79.8234,  98.1092],
        [123.3970, 137.8231],
        [ 22.8227,  40.2439],
        [ 96.8768, 112.7868]], grad_fn=<AddBackward0>)

In [34]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

We can see that the predictions are quite close to the target variables, and we can get even better results by training for a few more epochs.

## Linear regression using PyTorch built-ins
The model and training process above were implemented using basic matrix operations. But sice this is such a common pattern, PyTorch has several built-in functions and classes to make it easy to create and train models.

We begin by importing torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [35]:
import torch.nn as nn

As before, we represent the inputs and targets and matrices.

In [37]:
# Input (temp, rainfall, humidity)
inputs = np.array([
    [73,67,43],[91,88,64],[87,134,58],
    [102,43,37],[69,96,70],[73,67,43],
    [91,88,64],[87,134,58],[102,43,37],
    [69,96,70],[73,67,43],[91,88,64],
    [87,134,58],[102,43,37],[69,96,70]
],dtype = 'float32'
)

# Targets (apples,oranges)
targets = np.array([
    [56,70],[81,101],[119,133],
    [22,37],[103,119],[56,70],
    [81,101],[119,133],[22,37],
    [103,119],[56,70],[81,101],
    [119,133],[22,37],[103,119]
],dtype = 'float32'
)

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

We are using 15 training examples this time, to illustrate how to work with large datasets in small batches. (This is done because sometimes we may have millions of data).

### Dataset and DataLoader
We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

Note: 
    These utilities are used to easily work in batches.
    

In [38]:
from torch.utils.data import TensorDataset

In [39]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3] 

# The above code picks the first 3 rows of the input and output data

# train_ds[[1,3,5,7]] ,this allows us to acces slices of the dataset

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation( [0:3] in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [41]:
from torch.utils.data import DataLoader

In [42]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)

The data loader is typically used in a for-in loop. Let's look at an example.

In [43]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

# for xb, yb in train_dl:
#     print('batch:')
#     print(xb)
#     print(yb)

    
# This creates 5 rows of inputs and 5 rows or outputs after shuffling

tensor([[ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 87., 134.,  58.]])
tensor([[ 56.,  70.],
        [ 22.,  37.],
        [ 81., 101.],
        [119., 133.],
        [119., 133.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If shuffle is set True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

## nn.Linear
Instead of initializing the weights and biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [44]:
# Define model
model = nn.Linear(3,2)  # Here we specify the number of inputs and outputs
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.5105, -0.4760, -0.1162],
        [ 0.4568, -0.0160, -0.0865]], requires_grad=True)
Parameter containing:
tensor([-0.4913, -0.2200], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [46]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.5105, -0.4760, -0.1162],
         [ 0.4568, -0.0160, -0.0865]], requires_grad=True),
 Parameter containing:
 tensor([-0.4913, -0.2200], requires_grad=True)]

We can use the model to generate predictions in the exact same way as before

In [47]:
# Generate predictions
preds = model(inputs) 
preds

# Note: Earlier it was a model but now it is an object

tensor([[ -0.1090,  28.3356],
        [ -3.3553,  34.4059],
        [-26.5941,  32.3593],
        [ 26.8171,  42.4875],
        [-19.0923,  23.7086],
        [ -0.1090,  28.3356],
        [ -3.3553,  34.4059],
        [-26.5941,  32.3593],
        [ 26.8171,  42.4875],
        [-19.0923,  23.7086],
        [ -0.1090,  28.3356],
        [ -3.3553,  34.4059],
        [-26.5941,  32.3593],
        [ 26.8171,  42.4875],
        [-19.0923,  23.7086]], grad_fn=<AddmmBackward>)

## Loss function
Instead of defining a loss function manually, we can use built-in loss function mse_loss.

In [49]:
# Import nn.functional
import torch.nn.functional as F

The nn.functional package contains many useful functions and several other utilities.

In [50]:
# Define loss function
loss_fn = F.mse_loss

In [48]:
?nn.Linear # Documentation

In [51]:
# Computinf the loss for the current predictions of our model

loss = loss_fn(model(inputs),targets)
print(loss)

tensor(7180.1216, grad_fn=<MseLossBackward>)


## Optimizer
Insead of manually manipulating the model's weights and biases using gradients, we can use the optimizer optim.SGD. SGD stands for stochastic gradient descent. It is called stochastic because samples are selected in bathces(often with random shuffling) instead of as a single group.

In [52]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified(subtracted in this case).

## Train the model
We are now ready to train the model. We'll follow the exact same processto implement gradient descent:
    1. Generate predictions
    2. Calculate the loss
    3. Compute gradients w.r.t the weights and biases
    4. Adjust the weights by subtracting a small quantity
    5. Reset the gradients to zero
   
The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function fir which trains the model for a given number of epochs

In [58]:
# Utility function to train the model

def fit(num_epochs, model, loss_fn, opt):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for xb,yb in train_dl:
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
            
        # Print the progress
        if(epoch+1) %10 ==0: # Every 10th epoch we get message (i.e every batch)
            print('Epoch [{}/{}], Loss:{:.4f}'.format(epoch+1, num_epochs,loss.item()))

Note:
    * We use the data loader defined earlier to get batches of data for every iteration.
    * Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update, and opt.zero_grad to reset the gradients to zero.
    * We've also added a log statement which prints the loss from the last batch of data for every 10th epoch, to track the progress of training loss.item returns the actual value stores in the loss tensor.
   

In [59]:
# Training model for 100 epochs

fit(100,model,loss_fn,opt)

Epoch [10/100], Loss:745.7094
Epoch [20/100], Loss:249.9146
Epoch [30/100], Loss:329.1766
Epoch [40/100], Loss:194.6701
Epoch [50/100], Loss:116.7178
Epoch [60/100], Loss:115.0035
Epoch [70/100], Loss:98.5894
Epoch [80/100], Loss:32.2460
Epoch [90/100], Loss:39.9321
Epoch [100/100], Loss:41.6164


Now generating predictions using our model and verifying that they're close to our targets.

In [60]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.7253,  71.6962],
        [ 81.1619,  97.6718],
        [119.2787, 137.8428],
        [ 29.8480,  44.6616],
        [ 94.8372, 109.3945],
        [ 58.7253,  71.6962],
        [ 81.1619,  97.6718],
        [119.2787, 137.8428],
        [ 29.8480,  44.6616],
        [ 94.8372, 109.3945],
        [ 58.7253,  71.6962],
        [ 81.1619,  97.6718],
        [119.2787, 137.8428],
        [ 29.8480,  44.6616],
        [ 94.8372, 109.3945]], grad_fn=<AddmmBackward>)

In [61]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])